In [9]:
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
import os
FOLDER_NAME = '../dataset'

CLASSES = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']

In [10]:
# Create label map
label_map = {}
for (root, folders, files) in os.walk(FOLDER_NAME):
    for foldername in folders:
        if foldername in CLASSES:
            label_map[foldername] = CLASSES.index(foldername)

print(label_map)

{'a': 0, 'b': 1, 'c': 2, 'd': 3, 'e': 4, 'f': 5, 'g': 6, 'h': 7, 'i': 8, 'j': 9, 'k': 10, 'l': 11, 'm': 12, 'n': 13, 'o': 14, 'p': 15, 'r': 17, 's': 18, 't': 19, 'u': 20, 'v': 21, 'w': 22, 'x': 23, 'y': 24, 'z': 25}


In [11]:
sequence, label = [], []
target_length = 14
for (root, folders, files) in os.walk(FOLDER_NAME):
    for filename in files:
        file_path = os.path.join(os.path.relpath(
            root, FOLDER_NAME), filename)
        if (filename.endswith('.npy') and os.path.split(file_path)[0] in CLASSES):
            res = np.load(f'{FOLDER_NAME}/{file_path}')
            for _ in range(target_length-res.shape[0]):
                res = np.vstack((res, res[-1, :]))
            sequence.append(np.array(res))
            label.append(label_map[os.path.basename(root[-1])])

print(np.array(sequence).shape)
print(np.array(label).shape)

(552, 14, 1662)
(552,)


In [21]:
# FOR DATA DUPLICATION
sequence = np.concatenate([sequence, sequence, sequence, ], axis=0)
label = np.concatenate([label, label, label], axis=0)

print(np.array(sequence).shape)
print(np.array(label).shape)


(1656, 14, 1662)
(1656,)


In [22]:
X_train, X_test, y_train, y_test = train_test_split(np.array(sequence), tf.keras.utils.to_categorical(
    np.array(label).astype(int), num_classes=np.array(CLASSES).shape[0], dtype='float32'), test_size=0.05)

print(X_train.shape, X_test.shape)


(1573, 14, 1662) (83, 14, 1662)


In [23]:
from keras.models import Sequential
from keras.layers import LSTM, Dense
from keras.callbacks import TensorBoard


In [24]:
log_dir = os.path.join('Logs')
tb_callback = TensorBoard(log_dir=log_dir)


In [25]:
model = Sequential()
model.add(LSTM(64, return_sequences=True,
          activation='tanh', input_shape=(14, 1662)))
model.add(LSTM(128, return_sequences=True, activation='tanh'))
model.add(LSTM(64, return_sequences=False, activation='tanh'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(np.array(CLASSES).shape[0], activation='softmax'))


In [26]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['categorical_accuracy'])


In [27]:
tf.config.list_physical_devices('GPU')


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [28]:
model.fit(X_train, y_train, epochs=200, callbacks=[tb_callback])


Epoch 1/200
50/50 [==============================] - 6s 33ms/step - loss: 3.2276 - categorical_accuracy: 0.0509
Epoch 2/200
50/50 [==============================] - 1s 14ms/step - loss: 3.0727 - categorical_accuracy: 0.1036
Epoch 3/200
50/50 [==============================] - 1s 13ms/step - loss: 2.7323 - categorical_accuracy: 0.1310
Epoch 4/200
50/50 [==============================] - 1s 13ms/step - loss: 2.5706 - categorical_accuracy: 0.1284
Epoch 5/200
50/50 [==============================] - 1s 14ms/step - loss: 2.5226 - categorical_accuracy: 0.1214
Epoch 6/200
50/50 [==============================] - 1s 14ms/step - loss: 2.5029 - categorical_accuracy: 0.1316
Epoch 7/200
50/50 [==============================] - 1s 13ms/step - loss: 3.2209 - categorical_accuracy: 0.0668
Epoch 8/200
50/50 [==============================] - 1s 13ms/step - loss: 3.1806 - categorical_accuracy: 0.0515
Epoch 9/200
50/50 [==============================] - 1s 13ms/step - loss: 3.1552 - categorical_accuracy:

In [29]:
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm_3 (LSTM)               (None, 14, 64)            442112    
                                                                 
 lstm_4 (LSTM)               (None, 14, 128)           98816     
                                                                 
 lstm_5 (LSTM)               (None, 64)                49408     
                                                                 
 dense_3 (Dense)             (None, 64)                4160      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 26)                858       
                                                                 
Total params: 597,434
Trainable params: 597,434
Non-tr

In [20]:
res = model.predict(X_test)

1/1 [==============================] - 1s 1s/step


In [30]:
import numpy as np
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Assume X_test and y_test are your test dataset
# Make predictions
y_pred = model.predict(X_test)

# Convert predictions to class labels
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)  # if y_test is one-hot encoded

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred_classes)

# Calculate precision, recall, and F1 score for each class
precision = precision_score(y_true, y_pred_classes, average='weighted')
recall = recall_score(y_true, y_pred_classes, average='weighted')
f1 = f1_score(y_true, y_pred_classes, average='weighted')

# Print the results
print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1 Score: {f1}")

# Detailed classification report
report = classification_report(y_true, y_pred_classes)
print(report)


3/3 [==============================] - 1s 13ms/step
Accuracy: 0.8674698795180723
Precision: 0.9023427041499331
Recall: 0.8674698795180723
F1 Score: 0.8677832542290372
              precision    recall  f1-score   support

           0       0.83      1.00      0.91         5
           1       1.00      1.00      1.00         1
           2       0.80      1.00      0.89         4
           3       1.00      1.00      1.00         2
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         2
           6       1.00      1.00      1.00         3
           7       1.00      1.00      1.00         2
           8       1.00      0.50      0.67         4
           9       1.00      1.00      1.00         2
          10       1.00      0.67      0.80         3
          11       0.78      1.00      0.88         7
          12       1.00      1.00      1.00         3
          13       1.00      1.00      1.00         4
          14       0.7

c:\Users\Kanzoon\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Kanzoon\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Kanzoon\miniconda3\envs\tf\lib\site-packages\sklearn\metrics\_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\Kanzoon\miniconda3\envs\tf\lib\site-packages\sklea

In [51]:
actions = np.array(CLASSES)
testing = 2
print(actions[np.argmax(res[testing])], actions[np.argmax(y_test[testing])])

l l


In [52]:
model.save('action.h5')